<a href="https://colab.research.google.com/github/DavoII/handwritten_letters_recognition/blob/master/ARMENIAN_LETTERS_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from keras.utils import np_utils
from tensorflow.keras import optimizers
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [44]:
data = np.load('drive/My Drive/lowercase_agg_first_10/data.npy', encoding='bytes') 
labels = np.load('drive/My Drive/lowercase_agg_first_10/labels.npy', encoding='bytes') 
class_idx = np.load('drive/My Drive/lowercase_agg_first_10/label_classes.npy', encoding='bytes')

print(data.shape)
print(labels.shape)

(46016, 64, 64)
(46016, 10)


In [45]:
number_classes = len(class_idx)
print('number of classes', number_classes)

number of classes 10


In [0]:
x_train, x_test, y_train, y_test = train_test_split(
    data, labels,
    test_size=0.2,
    random_state=42,
    shuffle=True,
    stratify=labels
    )

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

In [0]:
(img_shape_w, img_shape_h) = x_train[0].shape
x_train = x_train.reshape(x_train.shape[0], img_shape_w, img_shape_h, 1)
x_test = x_test.reshape(x_test.shape[0], img_shape_w, img_shape_h, 1)

In [48]:
print('x_train', x_train.shape)
print('x_test', x_test.shape)
print('y_train', y_train.shape)
print('y_test', y_test.shape)

x_train (36812, 64, 64, 1)
x_test (9204, 64, 64, 1)
y_train (36812, 10)
y_test (9204, 10)


In [0]:
ly1_neurons = 128
ly2_neurons = 256
ly3_neurons = 518

kernel_size_1 = (5, 5)
kernel_size_2 = (5, 5)
kernel_size_3 = (3, 3)

dropout_1 = 0.1
dropout_2 = 0.2
dropout_3 = 0.5

pool_size_1 = (2, 2)
pool_size_2 = (2, 2)
pool_size_3 = (2, 2)

categories=number_classes

model = Sequential()

model.add(Conv2D(ly1_neurons, kernel_size_1, input_shape=x_train.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=pool_size_1))
model.add(Dropout(dropout_1))

model.add(Conv2D(ly2_neurons, kernel_size_2))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=pool_size_2))
model.add(Dropout(dropout_2))

model.add(Conv2D(ly3_neurons, kernel_size_3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=pool_size_3))
model.add(Dropout(dropout_3))

model.add(Flatten())
model.add(Dense( 512 ))
model.add(Activation('relu'))
model.add(Dense( categories ))
model.add(Activation('softmax'))

In [0]:
learning_rate = 0.0001
model.compile(optimizer=optimizers.Adam(
    learning_rate=learning_rate
), loss='categorical_crossentropy', metrics=['accuracy'])

In [51]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 60, 60, 128)       3328      
_________________________________________________________________
activation_15 (Activation)   (None, 60, 60, 128)       0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 26, 26, 256)       819456    
_________________________________________________________________
activation_16 (Activation)   (None, 26, 26, 256)       0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 256)      

In [52]:
model.fit(
    x_train,
    y_train,
    batch_size=120,
    epochs=10,
    validation_data=(x_test, y_test),
    verbose=1
    )

Epoch 1/10
307/307 [==============================] - 49s 160ms/step - loss: 0.8607 - accuracy: 0.7907 - val_loss: 0.5761 - val_accuracy: 0.8420
Epoch 2/10
307/307 [==============================] - 47s 154ms/step - loss: 0.4791 - accuracy: 0.8591 - val_loss: 0.3645 - val_accuracy: 0.8903
Epoch 3/10
307/307 [==============================] - 47s 154ms/step - loss: 0.3407 - accuracy: 0.8968 - val_loss: 0.3236 - val_accuracy: 0.9015
Epoch 4/10
307/307 [==============================] - 47s 155ms/step - loss: 0.2555 - accuracy: 0.9205 - val_loss: 0.2533 - val_accuracy: 0.9221
Epoch 5/10
307/307 [==============================] - 47s 154ms/step - loss: 0.2151 - accuracy: 0.9328 - val_loss: 0.2052 - val_accuracy: 0.9406
Epoch 6/10
307/307 [==============================] - 47s 154ms/step - loss: 0.1777 - accuracy: 0.9454 - val_loss: 0.1528 - val_accuracy: 0.9553
Epoch 7/10
307/307 [==============================] - 47s 154ms/step - loss: 0.1452 - accuracy: 0.9562 - val_loss: 0.1925 - val_ac

In [53]:
gen = ImageDataGenerator(rotation_range=12, width_shift_range=0.1, shear_range=0.3,
                        height_shift_range=0.1, zoom_range=0.1, data_format='channels_first')

batch_size = 512
batches = gen.flow(x_train, y_train, batch_size=batch_size)
test_batches = gen.flow(x_test, y_test, batch_size=batch_size)
steps_per_epoch = int(np.ceil(batches.n/batch_size))
validation_steps = int(np.ceil(test_batches.n/batch_size))

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3, or 4 channels on axis 1. However, it was passed an array with shape (36812, 64, 64, 1) (64 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3, or 4 channels on axis 1. However, it was passed an array with shape (9204, 64, 64, 1) (64 channels).
  str(self.x.shape[channels_axis]) + ' channels).')


In [54]:
model.fit(
    batches,
    epochs=15,
    validation_data=test_batches,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
    )

Epoch 1/15
72/72 [==============================] - 113s 2s/step - loss: 0.1933 - accuracy: 0.9398 - val_loss: 0.1535 - val_accuracy: 0.9539
Epoch 2/15
72/72 [==============================] - 109s 2s/step - loss: 0.1495 - accuracy: 0.9532 - val_loss: 0.1359 - val_accuracy: 0.9584
Epoch 3/15
72/72 [==============================] - 109s 2s/step - loss: 0.1417 - accuracy: 0.9559 - val_loss: 0.1825 - val_accuracy: 0.9433
Epoch 4/15
72/72 [==============================] - 108s 2s/step - loss: 0.1336 - accuracy: 0.9577 - val_loss: 0.1334 - val_accuracy: 0.9612
Epoch 5/15
72/72 [==============================] - 108s 2s/step - loss: 0.1351 - accuracy: 0.9577 - val_loss: 0.1345 - val_accuracy: 0.9588
Epoch 6/15
72/72 [==============================] - 109s 2s/step - loss: 0.1269 - accuracy: 0.9586 - val_loss: 0.1379 - val_accuracy: 0.9570
Epoch 7/15
72/72 [==============================] - 110s 2s/step - loss: 0.1252 - accuracy: 0.9602 - val_loss: 0.1326 - val_accuracy: 0.9574
Epoch 8/15
72